In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

!pip install datatable > /dev/null
import datatable as dt

#sample_submission = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
#test_df = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
#features_df = pd.read_csv('../input/jane-street-market-prediction/features.csv')
train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')
train_df = train_data_datatable.to_pandas()

In this notebook, I am visualising insights from discussions and notebook comment sections and adding my own two cents as I am getting started in this competition. Below I list the ressources I found most helpful:


# References and Other Insightful Ressources:

**Utility Function**

[1] [Discussion: Metric 'De-Anonymized'](https://www.kaggle.com/c/jane-street-market-prediction/discussion/199107) by [miguel perez](https://www.kaggle.com/miguelpm)

[2] [Discussion: Some initial thoughts](https://www.kaggle.com/c/jane-street-market-prediction/discussion/200495) by [marketneutral](https://www.kaggle.com/marketneutral)

[3] [Notebook: Understanding the Utility Score Function](https://www.kaggle.com/renataghisloti/understanding-the-utility-score-function) by [Renata Ghisloti Duarte de Souza](https://www.kaggle.com/renataghisloti) 

**Missing Values**

[4] [Discussion: Daily Patterns on Missing Data](https://www.kaggle.com/c/jane-street-market-prediction/discussion/201264) by [Alihan Zihna](https://www.kaggle.com/alihanz)

[5] [Notebook: NaN values depending on Time of Day](https://www.kaggle.com/tomwarrens/nan-values-depending-on-time-of-day) by [Tommaso Guerrini](https://www.kaggle.com/tomwarrens)

[6] [Discussion: Why it is so common to set missing value = -999?](https://www.kaggle.com/c/jane-street-market-prediction/discussion/201433) by [JieMeii](https://www.kaggle.com/jiemeii)

# Utility Score

First, let's have a look at the utility score. In the competition description it is describes as follows: 

> This competition is evaluated on a utility score. Each row in the test set represents a trading opportunity for which you will be predicting an `action` value, 1 to make the trade and 0 to pass on it. Each trade $j$ has an associated `weight` and `resp`, which represents a return.
>
> For each date $i$, we define:
>
> $p_i=\sum_j(weight_{ij}∗resp_{ij}∗action_{ij})$
>
> $t=\dfrac{\sum{p_i}}{\sqrt{\sum{p^2_i}}}*\sqrt{\dfrac{250}{|i|}}$ (close to **anualized Sharpe Ratio** if daily volatility of returns is estimated with mean zero with 250 trading days in a year[1]) 
>
> where $|i|$ is the number of unique dates in the test set. The utility is then defined as:
>
> $u=\min(\max(t,0),6)\sum{p_i}$ (A Sharpe Ratio higher than 6 is very unlikely, so it is capped at 6 [comment in 3])


For this utility score we aim to get as high of a value as possible - **the higher the better**. 

The variables for this utility score are `weight`, `resp`, `action`:
* `weight` is **given in both** the training and testing data
* `resp` is **only given in the training data** but not in the testing data
* `action` is **not given in both**

The **target variable** is `action` but we have to generate their values for the training data. In the threads, there is some discussion about whether setting the `action` values for **positive responses** will yield the highest utility score. Let's explore this hypothesis.

Note: The following plots only show a fraction of $|i|$ to demonstrate the tendencies. For the full 500 days, please check out version 2.

In [ ]:
i_ranges = 100 # Use 499 for full range

f, ax = plt.subplots(nrows=2, ncols=2, figsize=(14,14))
################
t_collected = np.zeros(i_ranges)
u_collected = np.zeros(i_ranges)
sum_p_collected = np.zeros(i_ranges)
sum_p_squared_collected = np.zeros(i_ranges)

for i_range in (range(i_ranges)):

    dates = np.linspace(0, i_range, i_range+1).astype(int)
    # Most commonly used
    threshold = 0
    
    p = np.zeros(len(dates))
    for i in dates:
        train_df_i = train_df[train_df.date == i]
        weight_i = train_df_i.weight
        resp_i = train_df_i.resp
        action_i = train_df_i.resp.apply(lambda x: 1 if x > threshold else 0)
        p[i] = np.sum(weight_i * resp_i * action_i)

    t = (np.sum(p) / np.sqrt(np.sum(p**2))) * np.sqrt(250/len(dates))
    u = np.minimum(np.maximum(t, 0), 6) * np.sum(p)
    
    t_collected[i_range] = t
    u_collected[i_range] = u
    sum_p_collected[i_range] = np.sum(p)
    sum_p_squared_collected[i_range] = np.sum(p**2)

ax[0, 0].plot(range(i_ranges),t_collected, 'b', label='Only Positive Response')
ax[0, 1].plot(range(i_ranges),u_collected, 'b', label='Only Positive Response')
ax[1, 0].plot(range(i_ranges),sum_p_collected, 'b', label='Only Positive Response')
ax[1, 1].plot(range(i_ranges),sum_p_collected/sum_p_squared_collected, 'b', label='Only Positive Response')

######################
t_collected = np.zeros(i_ranges)
u_collected = np.zeros(i_ranges)
sum_p_collected = np.zeros(i_ranges)
sum_p_squared_collected = np.zeros(i_ranges)

for i_range in (range(i_ranges)):

    dates = np.linspace(0, i_range, i_range+1).astype(int)
    # signal-to-noise or mean daily return divided by daily return std
    # mentioned here: https://www.kaggle.com/c/jane-street-market-prediction/discussion/199107
    threshold = train_df[train_df.date.isin(dates)].resp.mean()/train_df[train_df.date.isin(dates)].resp.std() 

    p = np.zeros(len(dates))
    for i in dates:
        train_df_i = train_df[train_df.date == i]
        weight_i = train_df_i.weight
        resp_i = train_df_i.resp
        action_i = train_df_i.resp.apply(lambda x: 1 if x > threshold else 0)
        p[i] = np.sum(weight_i * resp_i * action_i)

    t = (np.sum(p) / np.sqrt(np.sum(p**2))) * np.sqrt(250/len(dates))
    u = np.minimum(np.maximum(t, 0), 6) * np.sum(p)
    
    t_collected[i_range] = t
    u_collected[i_range] = u
    sum_p_collected[i_range] = np.sum(p)
    sum_p_squared_collected[i_range] = np.sum(p**2)

ax[0, 0].plot(range(i_ranges),t_collected, 'r', label='Greater Response than Signal-to-Noise')
ax[0, 1].plot(range(i_ranges),u_collected, 'r', label='Greater Response than Signal-to-Noise')
ax[1, 0].plot(range(i_ranges),sum_p_collected, 'r', label='Greater Response than Signal-to-Noise')
ax[1, 1].plot(range(i_ranges),sum_p_collected/sum_p_squared_collected, 'r', label='Greater Response than Signal-to-Noise')

######################

t_collected = np.zeros(i_ranges)
u_collected = np.zeros(i_ranges)
sum_p_collected = np.zeros(i_ranges)
sum_p_squared_collected = np.zeros(i_ranges)
# signal-to-noise or mean daily return divided by daily return std
# mentioned here: https://www.kaggle.com/c/jane-street-market-prediction/discussion/199107

threshold = train_df.resp.mean()/train_df.resp.std() 

for i_range in (range(i_ranges)):

    dates = np.linspace(0, i_range, i_range+1).astype(int)
    p = np.zeros(len(dates))
    for i in dates:
        train_df_i = train_df[train_df.date == i]
        weight_i = train_df_i.weight
        resp_i = train_df_i.resp
        action_i = train_df_i.resp.apply(lambda x: 1 if x > threshold else 0)
        p[i] = np.sum(weight_i * resp_i * action_i)

    t = (np.sum(p) / np.sqrt(np.sum(p**2))) * np.sqrt(250/len(dates))
    u = np.minimum(np.maximum(t, 0), 6) * np.sum(p)
    
    t_collected[i_range] = t
    u_collected[i_range] = u
    sum_p_collected[i_range] = np.sum(p)
    sum_p_squared_collected[i_range] = np.sum(p**2)

ax[0, 0].plot(range(i_ranges),t_collected, 'y', label='Greater Response than Overall Signal-to-Noise')
ax[0, 1].plot(range(i_ranges),u_collected, 'y', label='Greater Response than Overall Signal-to-Noise')
ax[1, 0].plot(range(i_ranges),sum_p_collected, 'y', label='Greater Response than Overall Signal-to-Noise')
ax[1, 1].plot(range(i_ranges),sum_p_collected/sum_p_squared_collected, 'y', label='Greater Response than Overall Signal-to-Noise')

######################

t_collected = np.zeros(i_ranges)
u_collected = np.zeros(i_ranges)
sum_p_collected = np.zeros(i_ranges)
sum_p_squared_collected = np.zeros(i_ranges)

for i_range in (range(i_ranges)):
    dates = np.linspace(0, i_range, i_range+1).astype(int)
    p = np.zeros(len(dates))
    for i in dates:
        train_df_i = train_df[train_df.date == i]
        weight_i = train_df_i.weight
        resp_i = train_df_i.resp

        # Selection all actions
        action_i = 1
        p[i] = np.sum(weight_i * resp_i * action_i)

    t = (np.sum(p) / np.sqrt(np.sum(p**2))) * np.sqrt(250/len(dates))
    u = np.minimum(np.maximum(t, 0), 6) * np.sum(p)
    
    t_collected[i_range] = t
    u_collected[i_range] = u
    sum_p_collected[i_range] = np.sum(p)
    sum_p_squared_collected[i_range] = np.sum(p**2)
    
ax[0, 0].plot(range(i_ranges),t_collected, 'g', label='All Actions')
ax[0, 1].plot(range(i_ranges),u_collected, 'g', label='All Actions')
ax[1, 0].plot(range(i_ranges),sum_p_collected, 'g', label='All Actions')
ax[1, 1].plot(range(i_ranges),sum_p_collected/sum_p_squared_collected, 'g', label='All Actions')

######################

t_collected = np.zeros(i_ranges)
u_collected = np.zeros(i_ranges)
sum_p_collected = np.zeros(i_ranges)
sum_p_squared_collected = np.zeros(i_ranges)

# Avoid calculations, everything is zero anyways

ax[0, 0].plot(range(i_ranges),t_collected, 'm', label='No actions')
ax[0, 1].plot(range(i_ranges),u_collected, 'm', label='No actions')
ax[1, 0].plot(range(i_ranges),sum_p_collected, 'm',  label='No actions')
ax[1, 1].plot(range(i_ranges),np.zeros(i_ranges), 'm',  label='No actions')

######################
ax[0, 0].plot(range(i_ranges),np.ones(i_ranges)*6, 'k', label='Max. t')

ax[0, 0].legend()
ax[0, 0].set_title('Annualized Sharpe Ratio t', fontsize=16)
ax[0, 0].set_xlabel('|i|', fontsize=16)
ax[0, 0].set_ylim([-3, 20])

ax[0, 1].legend()
ax[0, 1].set_title('Utility Score u', fontsize=16)
ax[0, 1].set_xlabel('|i|', fontsize=16)

ax[1, 0].legend()
ax[1, 0].set_title('$\sum p_i$', fontsize=16)
ax[1, 0].set_xlabel('|i|', fontsize=16)

ax[1, 1].legend()
ax[1, 1].set_title('$\sum p_i / \sum p_i^2$', fontsize=16)
ax[1, 1].set_xlabel('|i|', fontsize=16)
ax[1, 1].set_ylim([-0.01, 0.03])
plt.show()

According to [this thread](https://www.kaggle.com/c/jane-street-market-prediction/discussion/199107)
> The less volatile the returns achieved, the bigger score. So volatility is key in this competition.[1]


Above I have compared the utility scores over 250 days for four different action strategies:
*  <span style='color :magenta' >Strategy A: Taking no actions</span>
   
   Obviously, this will return a utility score of zero.
* <span style='color :green' >Strategy B: Taking all actions</span>

    In the long run yields a very low utility score similar to strategy A.
* <span style='color :blue' >Strategy C: Taking all actions with a return value greater than the daily volatility</span>
   
   The daily volatility of returns is estimated with mean zero. [1]
   Therefore, all actions with a positive return value (`resp > 0`) are taken.
   This seems to yielding the highest utility score.
   

* <span style='color :red' >Strategy D: Taking all actions with a return value greater than the daily signal-to-noise ratio</span> 

    The signal-to-noise ratio is the mean daily return divided by the standard deviation of the daily return (`resp.mean() / resp.std()` [1]
    Although the annualized Sharpe Ratio seems to be better than for strategy C, the cumulative returns are lower than in strategy C. In result, this ields a lower utility score than in strategy C. Additionally, in practice it might be very difficult to predict for each day.
    
* <span style='color :#FFCC66' >Strategy D: Taking all actions with a return value greater than the overall signal-to-noise ratio</span> 

    The overall signal-to-noise ratio is 0.01519. This is very close to strategy C's threshold of zero and a continuously constant threshold. Therefore, the behavior is very similar to strategy C but the overall utility score is lower.


So we need to find the balance between maximizing returns and maximizing the Sharpe Ratio.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14,6))
sns.boxplot(train_df.resp,ax=ax[0], )
ax[0].set_title(f"Distribution of Response \n(mean:{train_df.resp.mean():.4f}, std: {train_df.resp.std():.4f})", fontsize=14)
ax[0].set_xlim([-0.032, 0.032])

sns.scatterplot(data=train_df, x='weight', y='resp', ax=ax[1])
ax[1].set_title(f"Response over Weight", fontsize=14)
plt.show()

## Sharpe Ratio
Earlier, we learned that t is close to the **annualized Sharpe Ratio** if daily volatility of returns is estimated with mean zero with 250 trading days in a year[1]. 

Alright, but what exactly is the Sharpe ratio?

$\text{Sharpe Ratio} = \dfrac{R_p - R_f}{\sigma_p}$
* $R_p$ ... return of the investment
* $R_f$ ... risk-free rate
* $\sigma_p$ ... Standard deviation of returns


In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,6))
y_range = 10
y = np.linspace(0, y_range, y_range+1).astype(int)
sigma = (y-3)**2+6
ax.plot(sigma,y, 'black', label='No actions')

x_range = 30
x = np.linspace(0, x_range, x_range+1).astype(int)
R_f = 2
y = R_f + x*0.3
ax.plot(x, y, 'r', label='No actions')

ax.set_title('Return over Risk', fontsize=16)
ax.set_xlabel('Volatility (Risk)', fontsize=16)
ax.set_ylabel('Return', fontsize=16)
ax.set_xlim([0, 20])
ax.set_ylim([0, 8])

ax.scatter(10, 5, color='b')
ax.annotate('Optimal Portfolio', xy=(9, 4.5), fontsize=14, color='b')


ax.annotate('$R_f$', xy=(0, 1.8), fontsize=14, color='r')
ax.set_xticks([])
ax.set_yticks([])
plt.show()


For the standard deviation of returns
$\sigma_p = \sqrt{\dfrac{\sum{(X-\bar{X})^2}}{n}} = \sqrt{\dfrac{\sum{p_i^2}}{n}} = \dfrac{\sqrt{\sum{p_i^2}}}{\sqrt{n}}$
* $X = p_i$ ... returns
* $\bar{X} = 0$ ... estimate mean return as zero
* $n$ ... size of testset

$\text{Sharpe Ratio} = \dfrac{R_p - R_f}{\sigma_p} = \dfrac{(R_p - R_f)}{\sqrt{\sum{p_i^2}}}* \sqrt{n} \approx \dfrac{\sum{p_i}}{\sqrt{\sum{p_i^2}}}* \sqrt{n} $
* $R_p - R_f \approx \sum{p_i}$ ... return

Let's compare it to $t=\dfrac{\sum{p_i}}{\sqrt{\sum{p^2_i}}}*\sqrt{\dfrac{250}{|i|}}$


In [ ]:
train_df['p_i'] = train_df['weight'] * train_df['resp']
train_df['p_i^2'] = (train_df['p_i']**2)

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,6))


sns.scatterplot(data=train_df, x='p_i^2', y='p_i', ax=ax)

ax.set_title('Return over Risk', fontsize=16)
ax.set_xlabel('Volatility ($p_i^2$)', fontsize=16)
ax.set_ylabel('Return ($p_i$)', fontsize=16)
#ax.set_xlim([0, 20])
#ax.set_ylim([0, 8])

plt.show()

## Mathematical Approach
Alright, we have played around with different thresholds now and have seen that `action = resp > 0` so far yields the highest utility score.
Let's try the straight forward mathematical approach to get a definite answer on how to optimally set `action`.

We want to maximize: $u=\min(\max(t,0),6)\sum{p_i}$ 

For this we need to maximize t (w.r.t. to max. of 6) and $\sum{p_i}$.

To maximize $\sum{p_i}$, we need to sum up all positive $p_i$. Therefore `action = resp > 0` would be best here.


To maximize $t=\dfrac{\sum{p_i}}{\sqrt{\sum{p^2_i}}}*\sqrt{\dfrac{250}{|i|}}$, we have to find $p_i$, so that $\delta t = 0$. To achieve this, we also have to maximize $p_i$. 

**Therefore, `action = resp > 0` seems to yield the optimal utility function**. (Please correct me if you disagree.)



# Handling of Missing Values in Time Series

## Fundamentals
Handling missing values is an important preprocessing step. For time series it is especially important to carefully evaluate how to handle the missing values in relation to their neighouring values. Let's explore different options in the following example for a sine wave with some missing values:

In [ ]:
f, ax = plt.subplots(nrows=5, ncols=1, figsize=(16,20))

time        = np.arange(0, 13, 0.5)

amplitude   = np.sin(time)
amplitude_missing = np.array([ 0.        ,  0.47942554,  0.84147098,  0.99749499,  np.nan,  np.nan,  0.14112001, -0.35078323, -0.7568025 , -0.97753012,
                              -0.95892427, -0.70554033, -0.2794155 ,  0.21511999,  0.6569866 , 0.93799998,  0.98935825,  0.79848711,  0.41211849, 
                              np.nan, np.nan, -0.87969576, -0.99999021, -0.87545217, -0.53657292, -0.0663219 ])
ax[0].plot(time, amplitude, '--', color='k')
ax[0].plot(time, amplitude_missing, '-o', color='b')
ax[0].set_title('Missing Values in Time Series', fontsize=14)

ax[1].plot(time, amplitude, '--', color='k')
ax[1].plot(time, pd.Series(amplitude_missing).fillna(-999), '-o', color='b')
ax[1].set_title('Option 1: Fill NaN with Outlier (-999)', fontsize=14)

ax[2].plot(time, amplitude, '--', color='k')
ax[2].plot(time, pd.Series(amplitude_missing).fillna(pd.Series(amplitude_missing).mean()), '-o', color='b')
ax[2].set_title('Option 2: Fill NaN with mean', fontsize=14)

ax[3].plot(time, amplitude, '--', color='k')
ax[3].plot(time, pd.Series(amplitude_missing).ffill(), '-o', color='b')
ax[3].set_title('Option 3: Fill NaN with Last Value (ffill)', fontsize=14)

ax[4].plot(time, amplitude, '--', color='k')
ax[4].plot(time, pd.Series(amplitude_missing).interpolate(), '-o', color='b')
ax[4].set_title('Option 4: Fill NaN with Linearly Interpolated Value (interpolate)', fontsize=14)
plt.show()

* **Option 1: Fill NaN with Outlier**
    
    In this specific example filling the missing value with an outlier value such as -999 is not a good idea. 
    **However, many notebooks in this challenge have been using -999.** This is a valid question asked in [6]
    - should not be done with linear models, only with tree-based models
    - for XGBoost, which is a tree-based model,  missing=-999 
    - imputation
* **Option 2: Fill NaN with Mean Value**

    Also in this example, we can see that filling NaNs with the mean value is also not sufficient.
* **Option 3: Fill NaN with Last Value with `.ffill()`**

    Filling NaNs with the last value is already a little bit better in this case.
* **Option 4: Fill NaN with Linearly Interpolated Value with `.interpolate()`**

    Filling NaNs with the interpolated values is the best option in this small examlple but it requires knowledge of the neighouring values. This could be **not feasible for this challenge**.

In this short example, we saw that the way the missing values are handled affect the underlying information in the time series. In the following, we will analyse the missing values to evaluate how they should be handled ideally. 


## Patterns in Missing Values
The missing values follow specifc **patterns** and are **not missing randomly** [2, 4, 5] . Let's explore the patterns in the missing values first.

The below plots show the number of missing values for each trade for the first 14 days according to the `ts_id` order, which represents a time order:

In [ ]:
train_df['n_missing_vals'] = train_df.isna().sum(axis=1)

f, ax = plt.subplots(nrows=2, ncols=7, figsize=(24,8))

for r in range(2):
    for i in range(7):
        train_df[train_df.date == i+(7*r)].n_missing_vals.reset_index(drop=True).plot(ax=ax[r, i])
        ax[r, i].set_title(f'Day {i+(r*7)}')
        ax[r, i].set_ylim([-1, 80])
plt.suptitle('Daily Patterns of Missing Values', fontsize=16)
plt.show()

We can see that for most of the days, there is a similar behavior: 
In the beginning of the day, there are some features with missing values. In the middle of the day, there is a peak of missing values and the rest of the day has no missing values.
Furthermore, there are outliers, such as e.g. day 2 which shows a different behavior of missing values. 

Also, the amount of trades for each day varies. Therefore, we will do a quick detour and analyse some time-indicators before we continue with the analysis of the missing values. 

## Temporal Relationship
We know that `ts_id` indicates some time relationship between the trades in the form of an order. But there is no information about the time deltas between the trades. There is another promising feature that might indicate some time relationship between the trades: `feature_64`. 

Below plot shows `feature_64` over `ts_id` for the first five days. There is definitely a strongly relationship between them. We can also see that the outlier pattern from day 2 in above plot is related to `feature_64`. 

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(24,4))
sns.scatterplot(data=train_df[train_df.date.isin(range(5))], x='ts_id', y='feature_64', marker='.', linewidth=0, hue='date', palette='Set1')#.isna().sum(axis=1).plot(ax=ax)
ax.set_title(f'feature_64 over ts_id', fontsize=14)
plt.show()

If we create a new feature from `ts_id` by resetting `ts_id`back to zero at the beginning of each day, we can create a temporary dummy time feature called `ts_id_by_date`. Since the number of trades varies everyday, this will not be a good representation of the daily patterns as shown on the left below. However, if we plot the number of missing values over `feature_64`, we can see a pattern that is similar to the daily pattern as we saw in plot 'Daily Patterns of Missing Values'.

In this case, we could probably **use `feature_64` as a dummy timestamp.**

In [ ]:
# resetting `ts_id`back to zero at the beginning of each day
a = (train_df['date'] == train_df['date'].shift(1))
train_df['ts_id_by_date'] = a.cumsum()-a.cumsum().where(~a).ffill().fillna(0).astype(int)

f, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,4))

sns.scatterplot(data=train_df, x='ts_id_by_date', y='n_missing_vals',marker='.', linewidth=0, ax=ax[0])
ax[0].set_title('Number of Missing Values over Daily ts_id', fontsize=14)

sns.scatterplot(data=train_df, x='feature_64', y='n_missing_vals',marker='.', linewidth=0, ax=ax[1])
ax[1].set_title('Number of Missing Values over feature_64', fontsize=14)
plt.show()

Let's visualize some patterns over our new dummy timestamp `feature_64`.

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=5, figsize=(24,4))
sns.scatterplot(data=train_df, x='feature_64', y='resp', linewidth=0, marker='.', ax=ax[0])
ax[0].set_title('resp', fontsize=18)
for i in range(1,5):
    sns.scatterplot(data=train_df, x='feature_64', y=f'resp_{i}', linewidth=0, marker='.', ax=ax[i])
    ax[i].set_title(f'resp_{i}', fontsize=18)
plt.show()

Let's cluster the features according to their NaN patterns:

In [ ]:
features_with_missing_values = train_df.columns[train_df.isna().sum(axis=0) > 0]

features_with_missing_values = ['feature_55',
                                'feature_56', 
                                'feature_58', 
                                'feature_59', 
                                'feature_7', 'feature_8', 
                                'feature_11', 'feature_12',  
                                'feature_120', 'feature_121', 
                                'feature_122','feature_123', 
                                'feature_124', 'feature_125', 
                                'feature_126','feature_127', 
                                'feature_128', 'feature_129', 
                                'feature_3', 'feature_4','feature_44', 'feature_45',
                                'feature_17', 'feature_18', 'feature_27', 'feature_28',
                                'feature_21', 'feature_22', 'feature_31','feature_32',
                                'feature_9', 'feature_10','feature_19','feature_20', 'feature_29','feature_30','feature_73','feature_79','feature_85', 'feature_91','feature_97','feature_103','feature_109','feature_115',
                                'feature_13','feature_14','feature_23', 'feature_24', 'feature_33', 'feature_34','feature_75','feature_81','feature_87','feature_93', 'feature_99','feature_105', 'feature_111', 'feature_117',
                                'feature_15', 'feature_16', 'feature_25', 'feature_26','feature_35', 'feature_36', 'feature_76', 'feature_82', 'feature_88', 'feature_94','feature_100','feature_106', 'feature_112', 'feature_118',
                                'feature_72', 'feature_78', 'feature_84', 'feature_90','feature_96',  'feature_102', 'feature_108','feature_114',
                                'feature_74', 'feature_80','feature_86',  'feature_92','feature_98', 'feature_104', 'feature_110', 'feature_116',
                               ]
temp = train_df[features_with_missing_values].isna()
temp = temp.astype(int).values

n = np.zeros((len(features_with_missing_values), len(features_with_missing_values)))
for i in (range(len(features_with_missing_values))):
    for j in range(len(features_with_missing_values)):
        n[i, j] = (temp[:, i]==temp[:, j]).all()

del temp

n  = pd.DataFrame(columns = features_with_missing_values, data=n, index= features_with_missing_values)
del features_with_missing_values

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(20,20))
sns.heatmap(n, #mask=np.triu(np.ones_like(n, dtype=bool)), 
            cmap='Blues', linewidths=.5,)
ax.set_title('Clustered Features by Same NaN Pattern', fontsize=14)
plt.show()

As a next step, we will **check if there are any reoccuring patterns in the above found clusters**. For this, we will look at the missing values over the dummy time feature `feature_64`. I have divided `feature_64` into integer intervals for this. This is probably not the prettiest way to do this but sufficient for the purpose of analysing patterns. Each c Then I  take the mean of the `.isna()` for each cluster in every time interval. Each cluster is represented by its feature with the lowest feature number. E.g. `feature_9` represents the first big cluster containing `feature_9`, `feature_10`, `feature_20`, ....

This approach gives us following daily patterns of missing variables:

In [ ]:
temp = train_df[[f'feature_{c}' for c in [55, 56, 58, 59, 7, 11, 120, 122, 124, 126, 128, 3, 17, 21, 9, 13, 15, 72, 74 ]]].isna().astype(int)
temp['feature_64_int'] = train_df.feature_64.astype(int)

def color_green(val): 
    if val == 0: 
        return 'background-color: forestgreen'
patterns = temp.groupby('feature_64_int').mean()
patterns[['feature_9', 'feature_13', 'feature_15', 'feature_58','feature_59','feature_3',
           'feature_56',  'feature_122', 'feature_124', 'feature_126', 'feature_128',  
          'feature_55',  'feature_120',  'feature_11','feature_21','feature_74','feature_7','feature_17', 'feature_72']].style\
.background_gradient(cmap='RdYlGn_r', vmin=-0.2, vmax=1)\
.applymap(color_green).set_caption('Mean Missing Bit over Dummy Time ')

Now we can see the following clusters:
* Major Cluster 1:
    
    Only a few missing values around `feature_64` $\approx 1$ (Clusters with `feature_9`, `feature_13`, `feature_15`, see above heatmap 'Clustered Features by Same NaN Pattern')
       
    Only a few missing values around `feature_64` $\approx 58$ (Clusters with `feature_58`, see above heatmap)
* Major Cluster 2:

    Few missing values between 'late morning' and 'early afternoon' (Clusters with `feature_59`, `feature_3`, `feature_56`, `feature_122`, `feature_124`, `feature_126`, `feature_128`,, see above heatmap) 
    
* Major Cluster 3:

    Missing values in the 'morning' and few missing values after afterwards (Clusters with `feature_55`, `feature_120`, see above heatmap) 

* Major Cluster 4:

    Missing values in the 'morning', few missing values after afterwards, missing values at 'noon' (Clusters with  `feature_11`, `feature_21`, `feature_74`, see above heatmap) 

* Major Cluster 5:

    Missing values in the 'morning' and 'late morning' and missing values at 'noon' (Clusters with  `feature_7`, `feature_17`, `feature_72`, see above heatmap) 